In [1]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
import sagemaker
import boto3
# from time import gmtime, strftime
from datetime import datetime

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
region = boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sm_role = sagemaker.get_execution_role()
# Feel free to use your own role here
# sm_role = "arn:aws:iam::123456789012:role/sagemaker-custom-role"
print(f"Using Role: {sm_role}")
s3_bucket = sm_session.default_bucket()
print(f"Will use bucket '{s3_bucket}' for storing all resources related to this notebook")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using Role: arn:aws:iam::287758680514:role/service-role/AmazonSageMaker-ExecutionRole-20231027T004238
Will use bucket 'sagemaker-us-east-1-287758680514' for storing all resources related to this notebook


In [18]:
# from sagemaker import image_uris
#, model_uris, script_uris, hyperparameters
# from sagemaker.model import Model
# from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_id, model_version = ('mxnet-semseg-fcn-resnet101-coco', '*')
# endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")

endpoint_name = 'jumpstart-example-infer-mxnet-semseg-fc-2023-10-28-23-35-59-368'

In [4]:
bucket_prefix = "DEMO-async-inference"
resource_name = "AsyncInferenceDemo-{}-{}"

In [20]:
from sagemaker import image_uris

In [21]:
model_name = 'jumpstart-example-infer-mxnet-semseg-fc-2023-10-28-03-29-08-394'

In [22]:

# async_predictor = model_predictor.deploy(
#     async_inference_config=async_config,
#     instance_type="ml.m5.xlarge",
#     initial_instance_count=1,
#     endpoint_name=endpoint_name,
# )

In [23]:
endpoint_name = 'jumpstart-example-infer-mxnet-semseg-fc-2023-10-28-23-35-59-368'

In [24]:
jumpstart_assets_s3_bucket = f"jumpstart-cache-prod-{aws_region}"
pedestrian_img_key_prefix = "inference-notebook-assets"
pedestrian_img = "img_pedestrian.png"

boto3.client("s3").download_file(
    jumpstart_assets_s3_bucket, f"{pedestrian_img_key_prefix}/{pedestrian_img}", pedestrian_img
)

In [24]:
import json


def query(model_predictor, image_file_name):
    """Query the model predictor."""

    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()

    query_response = model_predictor.predict(
        input_img_rb,
        {
            "ContentType": "application/x-image",
            "Accept": "application/json;verbose",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return predictions as well as the set of all labels and object labels present in the image."""
    response_dict = json.loads(query_response)
    return response_dict["predictions"], response_dict["labels"], response_dict["image_labels"]


## testing async predictor for semantec segmentation Nov 3

In [6]:
jumpstart_assets_s3_bucket = "datascience-sagemaker-mk"
pedestrian_img_key_prefix = "image-processing"
pedestrian_img = "profilePic.jpg"

input_img_s3 =  f"s3://{jumpstart_assets_s3_bucket}/{pedestrian_img_key_prefix}/{pedestrian_img}"

input_img_s3 = "https://sagemaker-us-east-1-287758680514.s3.amazonaws.com/DEMO-async-inference/input/img_pedestrian.png"

In [9]:
sm_runtime = boto_session.client("sagemaker-runtime")

# image_file_name = "img_pedestrian.png"

# with open(image_file_name, "rb") as file:
#         input_img_rb = file.read()

        
# sm_runtime.invoke_endpoint_async(EndpointName='jumpstart-example-infer-mxnet-semseg-fc-2023-10-28-23-35-59-368', Body=input_img_rb,
#                           ContentType="application/x-image",
#                            Accept="application/json;verbose"
#         )
# async_resp = sm_runtime.invoke_endpoint_async(EndpointName='jumpstart-example-infer-mxnet-semseg-fc-2023-10-28-23-35-59-368', 
#                                  InputLocation=input_img_s3
#         )

async_resp = sm_runtime.invoke_endpoint_async(EndpointName='semantec-segmentation-async-with-sns', 
                                 InputLocation=input_img_s3,
                                              ContentType="application/x-image",
                                              Accept= "application/json;verbose"
        )

In [10]:
async_resp

{'ResponseMetadata': {'RequestId': '63e21c9d-3bc4-46cf-8fc1-ecca5681f891',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '63e21c9d-3bc4-46cf-8fc1-ecca5681f891',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/output/91d668cf-5f4a-4cdf-9e6f-1498610b0ab7.out',
   'x-amzn-sagemaker-failurelocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/failure/91d668cf-5f4a-4cdf-9e6f-1498610b0ab7-error.out',
   'date': 'Fri, 03 Nov 2023 22:51:30 GMT',
   'content-type': 'application/json',
   'content-length': '54',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/output/91d668cf-5f4a-4cdf-9e6f-1498610b0ab7.out',
 'FailureLocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/failure/91d668cf-5f4a-4cdf-9e6f-1498610b0ab7-error.out',
 'InferenceId': 'f8a9713e-377d-4eff-96db-19e557d850cc'}

In [25]:
async_resp

{'ResponseMetadata': {'RequestId': '26d7f340-268e-43be-85c2-261302fac3e7',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '26d7f340-268e-43be-85c2-261302fac3e7',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/output/f93e008d-469a-4ee0-8944-2fb2bc8858c7.out',
   'x-amzn-sagemaker-failurelocation': 's3://sagemaker-us-east-1-287758680514/async-endpoint-failures/jumpstart-example-infer-mxnet-semseg-fc-2023-10-1698536633-a322/f93e008d-469a-4ee0-8944-2fb2bc8858c7-error.out',
   'date': 'Mon, 30 Oct 2023 05:48:17 GMT',
   'content-type': 'application/json',
   'content-length': '54',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-287758680514/DEMO-async-inference/output/f93e008d-469a-4ee0-8944-2fb2bc8858c7.out',
 'FailureLocation': 's3://sagemaker-us-east-1-287758680514/async-endpoint-failures/jumpstart-example-infer-mxnet-semseg-fc-2023-10-1698536633-a322/f93e008d-469a-4ee0

In [16]:

query_response = query(base_model_predictor, pedestrian_img)

predictions, labels, image_labels = parse_response(query_response)
print("Objects present in the picture:", image_labels)

NameError: name 'base_model_predictor' is not defined

## trying with sync end pt

In [6]:


sm_runtime = boto_session.client("sagemaker-runtime")

image_file_name = "img_pedestrian.png"

with open(image_file_name, "rb") as file:
        input_img_rb = file.read()

        
resp = sm_runtime.invoke_endpoint(EndpointName='semantec-segmentation-sync-mxnet-semseg-2023-10-30-02-54-44-887', Body=input_img_rb,
                          ContentType="application/x-image",
                           Accept="application/json;verbose"
        )


In [15]:
resp

NameError: name 'resp' is not defined

In [12]:
for keys in resp.keys():
    print (resp[keys])

{'RequestId': '9b880701-afae-421c-b4e8-e15dc59ca33c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9b880701-afae-421c-b4e8-e15dc59ca33c', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 30 Oct 2023 05:22:02 GMT', 'content-type': 'application/json;verbose', 'content-length': '1529426', 'connection': 'keep-alive'}, 'RetryAttempts': 0}
application/json;verbose
AllTraffic


In [8]:
import json

In [14]:
jsresp = json.loads(resp['Body'].read().decode("utf-8"))

In [27]:
len(jsresp['predictions'])

536